In [ ]:
from bs4 import BeautifulSoup as bs
import requests

In [ ]:
url = 'https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&year1=2016&year2=2022&minmsrpsel=0&maxmsrpsel=0&city=0&hwy=0&comb=0&YearSel=2016-2022&make=&mclass=&vfuel=&vtype=&trany=&drive=&cyl=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=50&pageno=1&tabView=0'

In [ ]:
response = requests.get(url)

In [ ]:
response

<Response [200]>

In [ ]:
soup = bs(response.text, 'html.parser')

In [ ]:
soup

In [ ]:
## gdje se sve nalazi
tb = soup.find('tbody')

In [ ]:
## pronađi redove
rows = tb.find_all('tr', class_ = 'ymm-row')

In [ ]:
rows

In [ ]:
## pronađi imena
for name in rows:
  carfind = name.find('a')
  car = carfind.text
  print(car)

In [ ]:
## pronađi pogon i mijenjac
for name in rows:
  drivefind = name.find('span')
  drive = drivefind.text
  print(drive)

In [ ]:
## combined miles per galon
mpg_row = tb.find_all('td', class_ = 'mpg-comb')

for name in mpg_row:
  mpg = name.get_text()
  print(mpg)

In [ ]:
## anual fuel cost
fuel_row = tb.find_all('td', class_ = 'ann-fuel-cost')

for name in fuel_row:
  fuel = name.get_text()
  fuel = fuel[1:]
  print(fuel)

In [ ]:
import pandas as pd

In [ ]:
## first page into a dataframe
car_data = pd.DataFrame(columns=['Car Name', 'Manual/Motor', 'MPG', 'Annual Fuel Cost'])
for name, mpg_name, fuel_name in zip(rows, mpg_row, fuel_row):
    carfind = name.find('a')
    car = carfind.text

    drivefind = name.find('span')
    drive = drivefind.text

    mpg = mpg_name.get_text()

    fuel = fuel_name.get_text()[1:]

      # Append the data to the DataFrame
    car_data = car_data.append({'Car Name': car, 'Manual/Motor': drive, 'MPG': mpg, 'Annual Fuel Cost': fuel}, ignore_index=True)

In [ ]:
car_data

In [ ]:
## bring all the pages into 1 dataframe
car_infos = pd.DataFrame(columns=['Car Name', 'Manual/Motor', 'MPG', 'Annual Fuel Cost'])
for i in range(1,183):
  url = f'https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&year1=2016&year2=2022&minmsrpsel=0&maxmsrpsel=0&city=0&hwy=0&comb=0&YearSel=2016-2022&make=&mclass=&vfuel=&vtype=&trany=&drive=&cyl=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=50&pageno={i}&tabView=0'
  response = requests.get(url)
  soup = bs(response.text, 'html.parser')
  tb = soup.find('tbody')
  rows = tb.find_all('tr', class_ = 'ymm-row')
  mpg_row = tb.find_all('td', class_ = 'mpg-comb')
  fuel_row = tb.find_all('td', class_ = 'ann-fuel-cost')

  for name, mpg_name, fuel_name in zip(rows, mpg_row, fuel_row):
      carfind = name.find('a')
      car = carfind.text

      drivefind = name.find('span')
      drive = drivefind.text

      mpg = mpg_name.get_text()

      # fuel = fuel_name.get_text()[1:]
      fuel = fuel_name.get_text().strip()

      # Append the data to the DataFrame
      car_infos = car_infos.append({'Car Name': car, 'Manual/Motor': drive, 'MPG': mpg, 'Annual Fuel Cost': fuel}, ignore_index=True)


In [ ]:
car_infos

In [ ]:
car_infos.to_csv('cars.csv')